In [ ]:
# Setup dan Import
!pip install -q google-cloud-aiplatform
!pip install -q google-cloud-storage
!pip install -q scikit-learn pandas numpy joblib

import os
import json
from google.cloud import storage
from google.cloud import aiplatform
from google.cloud.aiplatform import Model
import joblib

In [ ]:
# Setup project
PROJECT_ID = "your-project-id"
BUCKET = "your-bucket"
REGION = "your-region"
ENDPOINT_NAME = "air-quality-model"

In [ ]:
# Initialize Vertex AI
aiplatform.init(project=PROJECT_ID, location=REGION)

In [ ]:
# Function untuk membuat model custom
def create_custom_model():
    """Membuat custom model untuk deployment"""
    
    def predict_fn(instances):
        """Function prediksi yang akan digunakan di endpoint"""
        outputs = []
        
        for instance in instances:
            # Preprocess input
            features = {
                'pm25_sensor': instance['pm25'],
                'pm10_sensor': instance['pm10'],
                'o3_sensor': instance['o3'],
                'co_sensor': instance['co'],
                'no2_sensor': instance['no2'],
                'temperature': instance['temperature'],
                'humidity': instance['humidity']
            }
            
            # Predict untuk setiap parameter
            calibrated = {}
            for param in ['pm25', 'pm10', 'o3', 'co', 'no2']:
                model = models[param]
                calibrated[f'{param}_calibrated'] = float(model.predict([list(features.values())])[0])
            
            outputs.append(calibrated)
        
        return outputs
    
    return predict_fn

In [ ]:
# Load models dari Cloud Storage
def load_models_from_gcs():
    storage_client = storage.Client()
    bucket = storage_client.bucket(BUCKET)
    models = {}
    
    for param in ['pm25', 'pm10', 'o3', 'co', 'no2']:
        blob = bucket.blob(f'models/{param}_model.joblib')
        blob.download_to_filename(f'/tmp/{param}_model.joblib')
        models[param] = joblib.load(f'/tmp/{param}_model.joblib')
    
    return models

In [ ]:
# Load models
models = load_models_from_gcs()

In [ ]:
# Create dan deploy model ke Vertex AI
def deploy_model():
    # Create model resource
    model = aiplatform.Model.upload(
        display_name=ENDPOINT_NAME,
        artifact_uri=f"gs://{BUCKET}/models/",
        serving_container_image_uri="gcr.io/cloud-aiplatform/prediction/sklearn-cpu.1-0:latest"
    )
    
    # Deploy model ke endpoint
    endpoint = model.deploy(
        machine_type="n1-standard-4",
        min_replica_count=1,
        max_replica_count=3
    )
    
    return endpoint

In [ ]:
# Deploy model
endpoint = deploy_model()

In [ ]:
# Test endpoint
def test_endpoint(endpoint):
    # Test data
    test_instance = {
        "pm25": 25.0,
        "pm10": 45.0,
        "o3": 0.035,
        "co": 1.2,
        "no2": 0.045,
        "temperature": 25.0,
        "humidity": 65.0
    }
    
    # Predict
    response = endpoint.predict([test_instance])
    
    print("Test prediction:")
    print(json.dumps(response, indent=2))

In [ ]:
# Test deployed model
test_endpoint(endpoint)

In [ ]:
# Setup monitoring
def setup_monitoring(endpoint):
    # Define metrics untuk monitoring
    monitoring_job = aiplatform.ModelDeploymentMonitoringJob.create(
        display_name=f"{ENDPOINT_NAME}-monitoring",
        endpoint=endpoint.resource_name,
        monitor_interval="hour",
        feature_monitoring_config={
            "target_field": "all",
            "feature_fields": ["pm25", "pm10", "o3", "co", "no2"]
        }
    )
    
    return monitoring_job

In [ ]:
# Setup monitoring untuk endpoint
monitoring_job = setup_monitoring(endpoint)

In [ ]:
print(f"""
Model Deployment Summary:
-----------------------
Endpoint Name: {ENDPOINT_NAME}
Endpoint URL: {endpoint.resource_name}
Model Version: {endpoint.model.version}
Monitoring Job: {monitoring_job.resource_name}
""")